Calculate Q for the following car lease details:

Q = fair_cost/ depreciation
fair_cost = depreciation - present value of tax benefits - present value of future reinvestment income on the depreciation
adjusted capitalized cost = 48990
residual value = 37257
depreciation = adjusted capitalized cost - residual value
21% federal tax rate
reinvestment rate 5%
discount rate 3%
2-year lease
support parametrizing MACRS years -- 5 and 7 years

create a python function returning a dictionary that contains the function parameters as well as fair_cost, the two present value results, and the MACRS rates used

In [44]:
import pprint

In [45]:
# claude

def calculate_lease_q_factor(adjusted_cap_cost, residual_value, property_class,
                           tax_rate, reinvestment_rate, discount_rate, lease_term):
    """
    Calculate Q factor using standard MACRS percentages applied to total depreciation

    Parameters:
    adjusted_cap_cost (float): Adjusted capitalized cost
    residual_value (float): Residual value at end of lease
    property_class (int): MACRS property class (5 or 7 years)
    tax_rate (float): Corporate tax rate (as decimal)
    reinvestment_rate (float): Rate at which depreciation can be reinvested (as decimal)
    discount_rate (float): Rate used for present value calculations (as decimal)
    lease_term (int): Lease term in years

    Returns:
    dict: Calculated Q factor and supporting details
    """
    # Total depreciation over lease period
    total_depreciation = adjusted_cap_cost - residual_value

    # Standard MACRS rates for 5 and 7 year property
    macrs_rates = {
        5: [0.20, 0.32, 0.192, 0.115, 0.115, 0.058],  # 5-year property
        7: [0.142, 0.249, 0.178, 0.127, 0.091, 0.091, 0.091, 0.031]  # 7-year property
    }

    if property_class not in macrs_rates:
        raise ValueError(f"Unsupported property class: {property_class}")

    # Get rates for the lease term
    lease_rates = macrs_rates[property_class][:lease_term]

    # Calculate annual depreciation amounts based on MACRS percentages
    annual_depreciation = [total_depreciation * rate for rate in lease_rates]

    # Calculate present value of tax benefits
    tax_benefits = []
    for year, dep_amount in enumerate(annual_depreciation):
        year_tax_benefit = dep_amount * tax_rate
        pv_factor = 1 / ((1 + discount_rate) ** (year + 1))
        present_value = year_tax_benefit * pv_factor
        tax_benefits.append(present_value)

    pv_tax_benefits = sum(tax_benefits)

    # Calculate present value of future reinvestment income
    # Assuming total depreciation is available at t=0 for reinvestment
    future_value = total_depreciation * ((1 + reinvestment_rate) ** lease_term - 1)
    pv_reinvestment = future_value / ((1 + discount_rate) ** lease_term)

    # Calculate Q
    fair_cost = total_depreciation - pv_tax_benefits - pv_reinvestment
    Q = fair_cost / total_depreciation

    return {
        'total_depreciation': total_depreciation,
        'annual_depreciation': annual_depreciation,
        'macrs_rates': lease_rates,
        'pv_tax_benefits': pv_tax_benefits,
        'pv_reinvestment': pv_reinvestment,
        'fair_cost': fair_cost,
        'Q_factor': Q,
        'detailed_data': {
            'year': list(range(1, lease_term + 1)),
            'depreciation': annual_depreciation,
            'tax_benefit': [dep * tax_rate for dep in annual_depreciation],
            'pv_tax_benefit': tax_benefits
        }
    }

In [46]:
result = calculate_lease_q_factor(
    adjusted_cap_cost=48990,
    residual_value=37257,
    property_class=5,
    tax_rate=0.21,
    reinvestment_rate=0.05,
    discount_rate=0.03,
    lease_term=2
)

pprint.pprint(result)

{'Q_factor': 0.7992647751908757,
 'annual_depreciation': [2346.6, 3754.56],
 'detailed_data': {'depreciation': [2346.6, 3754.56],
                   'pv_tax_benefit': [478.4330097087378, 743.196908285418],
                   'tax_benefit': [492.78599999999994, 788.4576],
                   'year': [1, 2]},
 'fair_cost': 9377.773607314544,
 'macrs_rates': [0.2, 0.32],
 'pv_reinvestment': 1133.5964746913003,
 'pv_tax_benefits': 1221.6299179941557,
 'total_depreciation': 11733}


In [47]:
#deepseek

def calculate_q(
    adjusted_capitalized_cost,
    residual_value,
    tax_rate,
    reinvestment_rate,
    discount_rate,
    lease_term,
    macrs_years
):
    """
    Calculate Q for a car lease based on the specified parameters.

    Parameters:
        adjusted_capitalized_cost (float): Adjusted capitalized cost of the car.
        residual_value (float): Residual value of the car at the end of the lease.
        tax_rate (float): Federal tax rate (e.g., 0.21 for 21%).
        reinvestment_rate (float): Reinvestment rate for depreciation (e.g., 0.05 for 5%).
        discount_rate (float): Discount rate for present value calculations (e.g., 0.03 for 3%).
        lease_term (int): Lease term in years.
        macrs_years (int): MACRS schedule (5 or 7 years).

    Returns:
        float: The calculated Q value.
    """
    # MACRS percentages for 5-year and 7-year schedules
    macrs_schedules = {
        5: [0.20, 0.32, 0.192, 0.1152, 0.1152, 0.0576],  # 5-year MACRS
        7: [0.1429, 0.2449, 0.1749, 0.1249, 0.0893, 0.0893, 0.0893, 0.0446]  # 7-year MACRS
    }

    # Validate MACRS years
    if macrs_years not in macrs_schedules:
        raise ValueError("MACRS years must be 5 or 7.")

    # Get MACRS percentages for the lease term
    macrs_percentages = macrs_schedules[macrs_years][:lease_term]

    # Step 1: Calculate Depreciation
    depreciation = adjusted_capitalized_cost - residual_value
    print(f"Depreciation: {depreciation}")

    # Step 2: Calculate Present Value of Tax Benefits
    pv_tax_benefits = 0
    for t in range(1, lease_term + 1):
        tax_benefit = depreciation * macrs_percentages[t - 1] * tax_rate
        pv_tax_benefits += tax_benefit / ((1 + discount_rate) ** t)
    print(f"Present Value of Tax Benefits: {pv_tax_benefits}")

    # Step 3: Calculate Future Reinvestment Income
    future_value = depreciation * (1 + reinvestment_rate) ** lease_term
    reinvestment_income = future_value - depreciation
    pv_reinvestment_income = reinvestment_income / ((1 + discount_rate) ** lease_term)
    print(f"Present Value of Reinvestment Income: {pv_reinvestment_income}")

    # Step 4: Calculate Q
    fair_cost = depreciation - pv_tax_benefits - pv_reinvestment_income
    Q = fair_cost / depreciation

    result = {
        'total_depreciation': depreciation,
        'macrs_rates': macrs_percentages,
        'pv_tax_benefits': pv_tax_benefits,
        'pv_reinvestment': pv_reinvestment_income,
        'fair_cost': fair_cost,
        'Q_factor': Q
    }
    return result


In [48]:
# Execute the function with the supplied parameters
q_value = calculate_q(
    adjusted_capitalized_cost=48990,
    residual_value=37257,
    tax_rate=0.21,
    reinvestment_rate=0.05,
    discount_rate=0.03,
    lease_term=2,
    macrs_years=5
)

pprint.pprint(f"results: {q_value}")

Depreciation: 11733
Present Value of Tax Benefits: 1221.6299179941557
Present Value of Reinvestment Income: 1133.5964746912996
("results: {'total_depreciation': 11733, 'macrs_rates': [0.2, 0.32], "
 "'pv_tax_benefits': 1221.6299179941557, 'pv_reinvestment': "
 "1133.5964746912996, 'fair_cost': 9377.773607314544, 'Q_factor': "
 '0.7992647751908757}')


In [49]:
# chatgpt

import numpy as np
import pprint

def calculate_q_factor(adjusted_capitalized_cost, residual_value, tax_rate, discount_rate, reinvestment_rate, lease_term, macrs_years):
    """
    Calculate the Q-factor for a car lease based on given parameters.

    Parameters:
        adjusted_capitalized_cost (float): Adjusted capitalized cost of the lease.
        residual_value (float): Residual value of the vehicle at lease end.
        tax_rate (float): Federal tax rate (as a decimal).
        discount_rate (float): Discount rate used for present value calculations (as a decimal).
        reinvestment_rate (float): Rate of return on reinvested depreciation (as a decimal).
        lease_term (int): Lease duration in years.
        macrs_years (int): Number of years for MACRS depreciation (either 5 or 7).

    Returns:
        dict: A dictionary containing Q-factor, fair cost, depreciation, tax benefits, reinvestment interest, and MACRS years.
    """

    # Calculate depreciation
    depreciation = adjusted_capitalized_cost - residual_value

    # MACRS depreciation schedules
    macrs_schedules = {
        5: np.array([0.20, 0.32, 0.192, 0.1152, 0.1152, 0.0576]),
        7: np.array([0.1429, 0.2449, 0.1749, 0.1249, 0.0893, 0.0892, 0.0893, 0.0446])
    }

    if macrs_years not in macrs_schedules:
        raise ValueError("MACRS years must be either 5 or 7.")

    macrs_schedule = macrs_schedules[macrs_years]

    # Compute PV of tax benefits (ONLY for the lease term)
    pv_tax_benefits = sum((tax_rate * depreciation * pct) / ((1 + discount_rate) ** (i + 1)) for i, pct in enumerate(macrs_schedule[:lease_term]))

    # Compute PV of reinvestment interest
    future_value = depreciation * ((1 + reinvestment_rate) ** lease_term)
    pv_reinvestment_interest = (future_value - depreciation) / ((1 + discount_rate) ** lease_term)

    # Compute fair cost
    fair_cost = depreciation - pv_tax_benefits - pv_reinvestment_interest

    # Compute Q-factor
    Q = fair_cost / depreciation

    # Prepare results in a dictionary
    result = {
        "Q_factor": Q,
        "fair_cost": fair_cost,
        "depreciation": depreciation,
        "pv_tax_benefits": pv_tax_benefits,
        "pv_reinvestment_interest": pv_reinvestment_interest,
        "macrs_years": macrs_years  # Add MACRS years to the output
    }

    # Pretty-print the dictionary
    pprint.pprint(result)

    return result


In [50]:
results_5yr = calculate_q_factor(
    adjusted_capitalized_cost=48990,
    residual_value=37257,
    tax_rate=0.21,
    discount_rate=0.03,
    reinvestment_rate=0.05,
    lease_term=2,
    macrs_years=5
)

results_7yr = calculate_q_factor(
    adjusted_capitalized_cost=48990,
    residual_value=37257,
    tax_rate=0.21,
    discount_rate=0.03,
    reinvestment_rate=0.05,
    lease_term=2,
    macrs_years=7
)


{'Q_factor': 0.7992647751908757,
 'depreciation': 11733,
 'fair_cost': 9377.773607314544,
 'macrs_years': 5,
 'pv_reinvestment_interest': 1133.5964746912996,
 'pv_tax_benefits': 1221.629917994156}
{'Q_factor': 0.8257722028466397,
 'depreciation': 11733,
 'fair_cost': 9688.785255999623,
 'macrs_years': 7,
 'pv_reinvestment_interest': 1133.5964746912996,
 'pv_tax_benefits': 910.618269309077}


In [51]:
!pip install numpy-financial

In [52]:
# gemini

import numpy_financial as npf

def calculate_lease_q(adjusted_capitalized_cost, residual_value, tax_rate, reinvestment_rate, discount_rate, lease_term, macrs_years):
    """Calculates the Q factor for a car lease."""

    depreciation = adjusted_capitalized_cost - residual_value

    if macrs_years == 5:
        macrs_rates = [0.20, 0.32, 0.192, 0.1152, 0.1152, 0.0576]
    elif macrs_years == 7:
        macrs_rates = [0.1429, 0.2449, 0.1749, 0.1249, 0.0893, 0.0892, 0.0893, 0.0446]
    else:
        raise ValueError("Invalid MACRS years. Choose 5 or 7.")

    # Calculate present value of tax benefits using npf.pv
    tax_benefits = []
    for i in range(lease_term):
        depreciation_amount = depreciation * macrs_rates[i] if i < len(macrs_rates) else 0
        tax_benefits.append(depreciation_amount * tax_rate)

    pv_tax_benefits = -npf.pv(discount_rate, range(1, lease_term + 1), 0, tax_benefits)  # Note the negative sign and handling of periods

    # Calculate present value of future reinvestment income using npf.pv
    reinvestment_income = []
    for i in range(lease_term):
        depreciation_amount = depreciation * macrs_rates[i] if i < len(macrs_rates) else 0
        reinvestment_income.append(depreciation_amount * tax_rate * reinvestment_rate)

    pv_reinvestment_income = -npf.pv(discount_rate, range(1, lease_term + 1), 0, reinvestment_income)  # Note the negative sign and handling of periods


    fair_cost = depreciation - pv_tax_benefits - pv_reinvestment_income
    Q = fair_cost / depreciation if depreciation != 0 else 0

    results = {
        "adjusted_capitalized_cost": adjusted_capitalized_cost,
        "residual_value": residual_value,
        "tax_rate": tax_rate,
        "reinvestment_rate": reinvestment_rate,
        "discount_rate": discount_rate,
        "lease_term": lease_term,
        "macrs_years": macrs_years,
        "macrs_rates": macrs_rates,
        "depreciation": depreciation,
        "pv_tax_benefits": pv_tax_benefits,
        "pv_reinvestment_income": pv_reinvestment_income,
        "fair_cost": fair_cost,
        "Q": Q
    }

    return results

In [53]:
# Example usage:
import pprint
lease_details = calculate_lease_q(
    adjusted_capitalized_cost=48990,
    residual_value=37257,
    tax_rate=0.21,
    reinvestment_rate=0.05,
    discount_rate=0.03,
    lease_term=2,
    macrs_years=5
)

pprint.pprint(lease_details)

lease_details_7 = calculate_lease_q(
    adjusted_capitalized_cost=48990,
    residual_value=37257,
    tax_rate=0.21,
    reinvestment_rate=0.05,
    discount_rate=0.03,
    lease_term=2,
    macrs_years=7
)

pprint.pprint(lease_details_7)

{'Q': array([0.95718447, 0.93349043]),
 'adjusted_capitalized_cost': 48990,
 'depreciation': 11733,
 'discount_rate': 0.03,
 'fair_cost': array([11230.64533981, 10952.6432463 ]),
 'lease_term': 2,
 'macrs_rates': [0.2, 0.32, 0.192, 0.1152, 0.1152, 0.0576],
 'macrs_years': 5,
 'pv_reinvestment_income': array([23.92165049, 37.15984541]),
 'pv_tax_benefits': array([478.43300971, 743.19690829]),
 'reinvestment_rate': 0.05,
 'residual_value': 37257,
 'tax_rate': 0.21}
{'Q': array([0.9694083, 0.9490994]),
 'adjusted_capitalized_cost': 48990,
 'depreciation': 11733,
 'discount_rate': 0.03,
 'fair_cost': array([11374.06759529, 11135.78322193]),
 'lease_term': 2,
 'macrs_rates': [0.1429,
                 0.2449,
                 0.1749,
                 0.1249,
                 0.0893,
                 0.0892,
                 0.0893,
                 0.0446],
 'macrs_years': 7,
 'pv_reinvestment_income': array([17.09201927, 28.43889419]),
 'pv_tax_benefits': array([341.84038544, 568.77788387])